# porting GROMOS topologies to openMM 

## importing `openMM_Factory`

In [1]:
import os

"""
import sys
SMArt_fd = os.path.abspath(os.getcwd())
for _ in range(2):
    SMArt_fd = os.path.split(SMArt_fd)[0]
sys.path.insert(0, SMArt_fd)
#"""
# if SMArt not in the path, uncomment the code above

import SMArt
from SMArt.md.openMM_Factory import openMM_Factory, mm
from SMArt.md import parse_top, parse_cnf, parse_imd


In [2]:
import numpy as np

from openmm import *
from openmm.app import *
from openmm.app.internal.unitcell import reducePeriodicBoxVectors
from openmm.unit import *


## parse GROMOS files

In [3]:
data_fd = os.path.abspath(os.path.join(os.path.split(SMArt.__file__)[0], '..', 'doc', 'some_data'))
data_fd = os.path.join(data_fd, 'gromos', 'GLY_LYSH_KMC')

top = parse_top(os.path.join(data_fd, 'GLY_LYSH_KMC.top'))
cnf = parse_cnf(os.path.join(data_fd, 'eq_GLY_LYSH_KMC.cnf'))
imd = parse_imd(os.path.join(data_fd, 'md.imd'))


## generate openMM_Factory instance

In [4]:
openMM_sys = openMM_Factory(top=top, MD_params=imd) # initialize

setting eps1 = 1.


In [5]:
## use solute only or add water molecules
flag_water = True
if flag_water:
    N_water = int((len(cnf.atoms) - len(top.atoms)) / 3)
else:
    N_water=0

## use native openMM electrostatic calculation or custom made one
flag_openMM_native_charges = False
if not flag_openMM_native_charges:
    # GROMOS custom energy
    openMM_sys.make_predefined_openmm_system(native_charges=False, N_water=N_water)
else:
    # native openMM charges + custom GROMOM energy for the rest
    openMM_sys.make_predefined_openmm_system(native_charges=True, N_water=N_water)


setting eps = 61.0


In [6]:
# cells above generate openMM system object
# stored in `system` attribute of the `openMM_Factory` instance
mm_sys = openMM_sys.system 

## generate openMM Simulation object

In [7]:
# generate openMM topology object (it could be made from a pdb file as well)
mm_top = openMM_sys.make_openmm_top()
# add the box information
mm_sys.setDefaultPeriodicBoxVectors(*reducePeriodicBoxVectors(np.diag(cnf.box.abc)))
mm_top.setPeriodicBoxVectors(reducePeriodicBoxVectors(np.diag(cnf.box.abc)))

# create integrator and simulation objects
integrator = NoseHooverIntegrator(300*kelvin, 0.1/picosecond, 0.002*picoseconds)
simulation = Simulation(mm_top, mm_sys, integrator)

# add coordinates and the box information to the simulation object
pos = [mm.vec3.Vec3(*at.coord) for at in cnf.atoms[:len(openMM_sys.top.atoms)]]
pos = mm.unit.quantity.Quantity(pos, mm.unit.nanometers)
simulation.context.setPositions(pos)
simulation.context.setPeriodicBoxVectors(*reducePeriodicBoxVectors(np.diag(cnf.box.abc)))


## use the `simulation` object

e.g. calculate energies 

In [8]:
for FG_name, i in openMM_sys.FG_map.items():
    st = simulation.context.getState(getPositions=True, getEnergy=True, groups={i})
    print('{:<35} {:<5}'.format(FG_name, i), st.getPotentialEnergy())

print('\nfull')
st = simulation.context.getState(getPositions=True, getEnergy=True)
print(st.getPotentialEnergy())


FG_BONDS                            0     0.0 kJ/mol
FG_ANGLES                           1     626.60546875 kJ/mol
FG_IMPROPERS                        2     15.926919937133789 kJ/mol
FG_DIHEDRALS                        3     48.74125671386719 kJ/mol
FG_NB_NATIVE_GROUP                  4     0.0 kJ/mol
FG_NB_LJ_coul_RF_RFC_CUSTOM         5     -84435.921875 kJ/mol
FG_NB_LJ_CUSTOM_excl_14             6     -8.948772430419922 kJ/mol
FG_NB_coul_RF_RFC_CUSTOM_14         7     406.5616760253906 kJ/mol
FG_NB_RF_RFC_CUSTOM_excl            8     150757.171875 kJ/mol
FG_NB_RFC_CUSTOM_self               9     -151014.40625 kJ/mol

full
-83604.25 kJ/mol
